In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os.path

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
def unpack_pickle(obj_name):
    file_path = "{0}.pkl".format(obj_name)
    n_bytes = 2**31
    max_bytes = 2**31 - 1
    data = bytearray(n_bytes)
    
    ## read
    bytes_in = bytearray(0)
    input_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
    loaded_data = pickle.loads(bytes_in)
    return loaded_data

In [3]:
X = unpack_pickle('X')

In [4]:
y = unpack_pickle('y')

In [5]:
X.shape, y.shape

((7730792, 73), (7730792,))

In [6]:
subm = pd.read_csv('../sample_submission.csv')

In [7]:
subm.head()

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0.5
1,00000176ac758d54827acd545b6315a5,0.5
2,0000019dcefc128c2d4387c1273dae1d,0.5
3,0000055553dc51b1295785415f1a224d,0.5
4,00000574cefffeca83ec8adf9285b2bf,0.5


In [8]:
test = unpack_pickle('../second_n_third/test')

In [9]:
test.shape, subm.shape

((7853253, 73), (7853253, 2))

In [10]:
X.columns.dtype == test.columns.dtype

True

In [11]:
test.select_dtypes(include='category').columns.tolist()

['Processor',
 'OsPlatformSubRelease',
 'SkuEdition',
 'Census_PrimaryDiskTypeName',
 'Census_PowerPlatformRoleName',
 'Census_OSArchitecture',
 'Census_OSInstallTypeName',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_GenuineStateName',
 'Census_ActivationChannel',
 'EngineVersion_1',
 'EngineVersion_2',
 'AppVersion_1',
 'AppVersion_2',
 'AvSigVersion_1',
 'Census_OSVersion_1']

In [12]:
dtypes = {}
for i in X.columns:
    dtypes[i] = str(X[i].dtype)

In [13]:
dtypes

{'AVProductStatesIdentifier': 'float32',
 'AVProductsInstalled': 'float16',
 'AppVersion': 'float64',
 'AppVersion_1': 'float64',
 'AppVersion_2': 'float64',
 'AppVersion_3': 'float64',
 'AvSigVersion': 'float64',
 'AvSigVersion_1': 'float64',
 'AvSigVersion_2': 'float64',
 'AvSigVersion_3': 'float64',
 'Census_ActivationChannel': 'float64',
 'Census_ChassisTypeName': 'float64',
 'Census_FirmwareManufacturerIdentifier': 'float16',
 'Census_FirmwareVersionIdentifier': 'float32',
 'Census_FlightRing': 'float64',
 'Census_GenuineStateName': 'float64',
 'Census_HasOpticalDiskDrive': 'int8',
 'Census_InternalBatteryNumberOfCharges': 'float32',
 'Census_InternalBatteryType': 'float64',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches': 'float16',
 'Census_InternalPrimaryDisplayResolutionHorizontal': 'float16',
 'Census_InternalPrimaryDisplayResolutionVertical': 'float16',
 'Census_IsAlwaysOnAlwaysConnectedCapable': 'float16',
 'Census_IsSecureBootEnabled': 'int8',
 'Census_IsTouchEnabled'

In [14]:
for i in dtypes:
    test[i] = test[i].astype(dtypes[i])

In [15]:
test.select_dtypes(include='category').columns.tolist()

[]

In [16]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import gc
gc.enable()

In [17]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'n_estimators': 4000,
    'colsample_bytree': 0.8, 
    'learning_rate': 0.6000000000000001, 
    'max_depth': 20,
    'metric': 'auc',
    'reg_alpha': 0.0, 
    'reg_lambda': 0.0, 
    'subsample': 0.8
}

In [18]:
sub=subm['MachineIdentifier'].to_frame()
sub['1']=0

In [19]:
sub.head()

,MachineIdentifier,1
0,0000010489e3af074adeac69c53e555e,0
1,00000176ac758d54827acd545b6315a5,0
2,0000019dcefc128c2d4387c1273dae1d,0
3,0000055553dc51b1295785415f1a224d,0
4,00000574cefffeca83ec8adf9285b2bf,0


In [20]:
kfold = 4

In [21]:
X = X.values
y = y.values

In [22]:
test = test.values

In [23]:
gc.collect()

21

In [24]:
skf = StratifiedKFold(n_splits=kfold, random_state=1300)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' lgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_eval = X[train_index], X[test_index]
    y_train, y_eval = y[train_index], y[test_index]
    dset = lgb.Dataset(X_train, label=y_train)
    dset_eval = lgb.Dataset(X_eval, label=y_eval)
    lgb_model = lgb.train(params, dset, 1000, dset_eval,
                          verbose_eval=100, early_stopping_rounds=1000)
    sub['1'] += lgb_model.predict(test, 
                        num_iteration=lgb_model.best_iteration) / (kfold)
     
gc.collect()

 lgb kfold: 1  of  4 : 


/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 1000 rounds.
[100]	valid_0's auc: 0.726947
[200]	valid_0's auc: 0.730011
[300]	valid_0's auc: 0.731222
[400]	valid_0's auc: 0.731733
[500]	valid_0's auc: 0.731946
[600]	valid_0's auc: 0.731846
[700]	valid_0's auc: 0.731481
[800]	valid_0's auc: 0.731233
[900]	valid_0's auc: 0.73082
[1000]	valid_0's auc: 0.730556
[1100]	valid_0's auc: 0.730202
[1200]	valid_0's auc: 0.729894
[1300]	valid_0's auc: 0.729511
[1400]	valid_0's auc: 0.729336
Early stopping, best iteration is:
[490]	valid_0's auc: 0.732017
 lgb kfold: 2  of  4 : 
Training until validation scores don't improve for 1000 rounds.
[100]	valid_0's auc: 0.726403
[200]	valid_0's auc: 0.729559
[300]	valid_0's auc: 0.73067
[400]	valid_0's auc: 0.731143
[500]	valid_0's auc: 0.731277
[600]	valid_0's auc: 0.73134
[700]	valid_0's auc: 0.731142
[800]	valid_0's auc: 0.730917
[900]	valid_0's auc: 0.73063
[1000]	valid_0's auc: 0.730364
Did not meet early stopping. Best iteration is:
[551]	valid_0

78

In [25]:
public_nffm_subm = pd.read_csv('nffm_submission.csv')

In [26]:
sub['2'] = public_nffm_subm['HasDetections']

In [27]:
sub['HasDetections'] = (sub['1'] + sub['2']) / 2

In [28]:
sub.head()

,MachineIdentifier,1,2,HasDetections
0,0000010489e3af074adeac69c53e555e,0.616117,0.528249,0.572183
1,00000176ac758d54827acd545b6315a5,0.675166,0.514046,0.594606
2,0000019dcefc128c2d4387c1273dae1d,0.605462,0.550244,0.577853
3,0000055553dc51b1295785415f1a224d,0.559104,0.360985,0.460044
4,00000574cefffeca83ec8adf9285b2bf,0.610959,0.486810,0.548884


In [29]:
sub = sub.drop(columns=['1', '2'])

In [30]:
sub.head()

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0.572183
1,00000176ac758d54827acd545b6315a5,0.594606
2,0000019dcefc128c2d4387c1273dae1d,0.577853
3,0000055553dc51b1295785415f1a224d,0.460044
4,00000574cefffeca83ec8adf9285b2bf,0.548884


In [31]:
sub.to_csv('sixth_submit.csv', index=False, float_format='%.5f')

Sixth submit. mean(CV after hyperopt new LGB params + with public NFFM 890 score predict) Lambda and alpha are now == 0.0, early_stopping_rounds=1000